In [ ]:
import pandas as pd
import numpy as np

In [ ]:
def cleanNewsDf(df):
  uniqueDates = df.groupby('Date')
  newsDic = {}
  for date,group in uniqueDates:
    for news in (group['News'].to_list()):
      if date in newsDic:
        newsDic[date][0] += news
      else:
        newsDic[date] = [news]
  
  dfDate = pd.DataFrame.from_dict(newsDic,'index')
  dfDate.columns = ['News']
  dfDate['Date'] = dfDate.index
  dfDate = dfDate.reindex(columns = ['Date', 'News'])
  dfDate = dfDate.reset_index(drop=True)
  dfDate = dfDate.replace(np.nan, ' ', regex=True)
  dfDate = dfDate.replace('b\"|b\'|\\\\|\\\"', '', regex=True)
  return dfDate

In [ ]:
redditNews    = pd.read_csv('/content/sample_data/RedditNews.csv')
redditNews = cleanNewsDf(redditNews)

In [ ]:
import nltk
nltk.download('vader_lexicon') # one time only (Valence Aware Dictionary and sEntiment Reasoner)
from nltk.sentiment.vader import SentimentIntensityAnalyzer
vader = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


/usr/local/lib/python3.6/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [ ]:
def addPolarityScores(df):
  negLst = []
  neuLst = []
  posLst = []
  comLst = []

  nrows = df.shape[0]
  for i in range(nrows):
    s = str(df.iloc[i,1])
    pScores = vader.polarity_scores(s)
    neg = pScores['neg']
    negLst.append(neg)
    neu = pScores['neu']
    neuLst.append(neu)
    pos = pScores['pos']
    posLst.append(pos)
    com = pScores['compound']
    comLst.append(com)

  df['neg'] = negLst
  df['neu'] = neuLst
  df['pos'] = posLst
  df['compound'] = comLst

  return df

In [ ]:
redditSentiment = addPolarityScores(redditNews)
redditSentiment

,Date,News,neg,neu,pos,compound
0,2008-06-08,Nim Chimpsky: The tragedy of the chimp who tho...,0.208,0.763,0.029,-0.9974
1,2008-06-09,United States quits Human Rights Council'Penta...,0.218,0.705,0.078,-0.9972
2,2008-06-10,"Oil shortage a myth, says industry insider'Isr...",0.212,0.748,0.040,-0.9943
3,2008-06-11,BBC uncovers lost Iraq billions'War Criminal G...,0.167,0.732,0.101,-0.9728
4,2008-06-12,Did US attack Pakistan last night? Pakistan fu...,0.150,0.770,0.080,-0.9681
...,...,...,...,...,...,...
2938,2016-06-27,Barclays and RBS shares suspended from trading...,0.151,0.759,0.090,-0.9683
2939,2016-06-28,"2,500 Scientists To Australia: If You Want To ...",0.137,0.763,0.100,-0.9633
2940,2016-06-29,Explosion At Airport In IstanbulYemeni former ...,0.209,0.715,0.076,-0.9968
2941,2016-06-30,Jamaica proposes marijuana dispensers for tour...,0.211,0.735,0.053,-0.9978


In [ ]:
redditSentiment.to_csv('djiaNewsSentiment.csv')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
